In [1]:
%pip install mediapipe opencv-python tensorflow pandas setuptools scikit-learn

You should consider upgrading via the '/Users/vikrantverma/Documents/mediapipe-form-tracker/py3-10/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hol = mp.solutions.holistic
#bicep_predict = tf.keras.models.load_model('bicep_curl_model.keras')
model = joblib.load('BicepBro.pkl')
#print(bicep_predict.summary())



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533 (2.09 KB)

 Trainable params: 177 (708.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 356 (1.39 KB)

None


In [3]:
def calculate_angles(pointA, pointB, pointC):
   # Vectors from point B (elbow) to points A (shoulder) and C (wrist)
    AB = np.array(pointA) - np.array(pointB)
    BC = np.array(pointC) - np.array(pointB)
    
    # Normalize vectors
    AB = AB / np.linalg.norm(AB)
    BC = BC / np.linalg.norm(BC)
    
    # Calculate the cosine of the angle using the dot product
    cosine_angle = np.dot(AB, BC)
    
    # Ensure the cosine value is within the valid range due to floating point errors
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    
    # Calculate the angle in degrees
    angle = np.arccos(cosine_angle)
    angle = np.degrees(angle)
    
    return angle



In [ ]:
cap = cv2.VideoCapture(0)

counter = 0
stage = None
time = 0
form_prediction = "Good"

with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
   while cap.isOpened():
      ret, frame = cap.read()

      '''
      the cv2 library uses bgr by default but mediapipe wants 
      frames in rgb format
      '''
      
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # more memory efficient because ensures that we don't need to make a copy of our image (array)
      image.flags.writeable = False 


      results = pose.process(image)

      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      try:
         landmarks = results.pose_landmarks.landmark
         real = results.pose_world_landmarks.landmark

         
         l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]

         r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]

         elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
         
         real_elbow = [real[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  real[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                  real[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
         
         real_hip = [real[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                  real[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                  real[mp_pose.PoseLandmark.LEFT_HIP.value].z]
         
         r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]

         wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

         angle = calculate_angles(l_shoulder[:2], elbow[:2], wrist)
         elbow_position_normalized = tuple(np.multiply(elbow[:2], [640,480]).astype(int))
         text_pos = (elbow_position_normalized[0] + 10, elbow_position_normalized[1] - 10)

         
         cv2.putText(
            image,
            str(int(angle)),
            text_pos,
            
            cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
         )
         
         if time % 10 == 0:
            try:
               form_prediction = model.predict(np.array([[l_shoulder[2], elbow[2], l_shoulder[1], r_shoulder[1], real_elbow[2], real_hip[2]]]))
               if form_prediction[0] == 0:
                  form_prediction = "Good"
               elif form_prediction[0] == 1:
                  form_prediction = "Lower your shoulder"
               elif form_prediction[0] == 2:
                  form_prediction = "Raise your shoulder"
               elif form_prediction[0] == 3:
                  form_prediction = "Keep your elbow next to your body"
               elif form_prediction[0] == 4:
                  form_prediction = "Lower your shoulder and keep elbow close"
               else:
                  form_prediction = "Raise shoulder and keep elbow close"
            except Exception as e:
               print(e)
               pass
         


         if angle > 160:
            stage = 'd'
         if angle <= 47 and stage == 'd':
            stage = 'u'
            counter += 1
      except Exception as e:
         print(e)
         pass

      cv2.putText(
            image,
            f"Reps: {str(counter)}",
            (200,200),
            cv2.FONT_HERSHEY_DUPLEX, 5, (255,255,255), 2, cv2.LINE_AA
         )

      cv2.putText(
            image,
            f"Technique: {form_prediction}",
            (0,480),
            cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2, cv2.LINE_AA
         )
      


      mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )

      cv2.imshow('Strongr v1', image)
      time += 1
      key = cv2.waitKey(10)

      if key==ord('q'):
         cv2.destroyAllWindows()
         break

   cv2.waitKey(1)
   cap.release()
   cv2.destroyAllWindows()




In [5]:
elbow

[0.7232216596603394, 0.9559380412101746, -0.5719205737113953]

In [6]:
wrist

[0.7144585847854614, 1.0940802097320557]